In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model_name = "Qwen/Qwen2.5-0.5B"

# Initialize model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [5]:
num1 = "323"
op = "*"
num2 = "68"

initial_prompt = f"""A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> RESULT_NUMBER </answer>. \nUser: Give me the answer of the following equation: {num1} {op} {num2}.\nAssistant: Let me solve this step by step.\n<think>"""

completion = "To find the product of 323 and 68, we can use the distributive property: 323 * 68 = 323 * (60 + 8) = (323 * 60) + (323 "

In [7]:
def get_n_single_tokens(prompt: str, n: int, temperature: float = 0.7) -> list:
    """Generate N single tokens from a prompt using specified temperature."""
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    generated_tokens = []
    
    for _ in range(n):
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=1,
                do_sample=True,
                temperature=temperature,
                pad_token_id=tokenizer.pad_token_id,
                num_return_sequences=1
            )
            
            new_token = outputs[0][len(inputs['input_ids'][0]):]
            decoded_token = tokenizer.decode(new_token)
            generated_tokens.append(decoded_token)
    
    return generated_tokens

In [9]:
prompt = initial_prompt + completion
n_generations = 50
temperature = 1.0

In [10]:
# Example usage
tokens = get_n_single_tokens(prompt, n=n_generations, temperature=temperature)
print("Generated tokens:")
for token in tokens:
    print(token)

Generated tokens:
5
8
 
8
 
 
8
0
8
8
3
8
8
8
8
 *
 which
1
8
 =
3
8
3
8
8
 times
1
8
8
8
 *
8
8
8
3
3
3
8
3
8
8
3
 *
3
8
8
 *
2
8
8
